# Multi-Convolutional Net for Sentiment Classification

This Conv Net performs sentiment analysis on the IMDB review dataset.

In [ ]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, concatenate
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers import Embedding, Conv1D, SpatialDropout1D, GlobalMaxPool1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#### Set Hyperparameters

In [ ]:
output_dir = 'model_output/multi-conv'

epochs = 4
batch_size = 128

n_dim = 64
n_unique_words = 5000
max_review_length = 400
pad_type = trunc_type = 'pre'
drop_embed = 0.2

n_conv_1 = n_conv_2 = n_conv_3 = 256
k_conv_1 = 3
k_conv_2 = 2
k_conv_3 = 4

n_dense = 256
dropout = 0.2

#### Load Data

In [ ]:
(X_train, y_train), (X_valid, y_valid) = imdb.load_data(num_words=n_unique_words)

#### Preprocess Data

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
X_valid = pad_sequences(X_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

#### Design Deep Net Architecture

In [ ]:
input_layer = Input(shape=(max_review_length,), dtype='int16', name='input')

embedding_layer = Embedding(n_unique_words, n_dim, input_length=max_review_length, name='embedding_1')(input_layer)
drop_embed_layer = SpatialDropout1D(drop_embed, name='drop_embed')(embedding_layer)

conv_1 = Conv1D(n_conv_1, k_conv_1, activation='relu', name='conv_1')(drop_embed_layer)
maxp_1 = GlobalMaxPool1D(name='maxp_1')(conv_1)

conv_2 = Conv1D(n_conv_2, k_conv_2, activation='relu', name='conv_2')(drop_embed_layer)
maxp_2 = GlobalMaxPool1D(name='maxp_2')(conv_2)

conv_3 = Conv1D(n_conv_3, k_conv_3, activation='relu', name='conv_3')(drop_embed_layer)
maxp_3 = GlobalMaxPool1D(name='maxp_3')(conv_3)

concat = concatenate([maxp_1, maxp_2, maxp_3])

dense_layer_1 = Dense(n_dense, activation='relu', name='dense_1')(concat)
drop_dense_layer_1 = Dropout(dropout, name='drop_dense_1')(dense_layer_1)
dense_layer_2 = Dense(n_dense, activation='relu', name='dense_2')(drop_dense_layer_1)
drop_dense_layer_2 = Dropout(dropout, name='drop_dense_2')(dense_layer_2)

predictions = Dense(1, activation='sigmoid', name='output')(drop_dense_layer_2)

model = Model(input_layer, predictions)


In [ ]:
model.summary()

#### Configure the Model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+'/weights.{epoch:02d}.hdf5')

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Train the Model

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_valid, y_valid))#, callbacks=[modelcheckpoint])

#### Evaluate

In [ ]:
#model.load_weights(output_dir+'/weights.01.hdf5')

In [ ]:
y_hat = model.predict_proba(X_valid)

In [ ]:
y_hat[0]

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat) * 100

In [ ]:
'{:0.2f}'.format(pct_auc)